In [1]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp -r /content/drive/MyDrive/"semester 8"/Kontur/. /content

In [4]:
from utils.data_processing import process_train_df,text_to_words_list
from utils.index_extraction import extract_word_position_from_texts
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from tqdm import tqdm

import torch
from torch.utils.data import Dataset,DataLoader

from transformers import AutoTokenizer, AutoModel
from nltk.lm import Vocabulary

from IPython.display import clear_output

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Model

## Data Processing

In [5]:
df_train = pd.read_json("dataset/train.json")
df_train = process_train_df(df_train)
df_train.head(4)

,id,text,label,text_output,answer_start,answer_end
0,809436509,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 6593.2...,1279,1343
1,854885310,ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запро...,обеспечение исполнения контракта,Поставщик должен предоставить обеспечение испо...,1222,1318
2,4382157,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1297,1343
3,184555082,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1304,1350


In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
text_list_of_words, text_all_words_list, word_position_list = text_to_words_list(list(df_train.text))
output_text_list_of_words, output_text_all_words_list, output_word_position_list = text_to_words_list(list(df_train.text_output))
output_word_position_list = extract_word_position_from_texts(word_position_list,text_list_of_words,output_text_list_of_words,df_train)

In [8]:
seq_lens = [len(text_list_of_words[i]) for i in range(len(text_list_of_words))]
max(seq_lens)

1015

In [9]:
len(text_list_of_words[0]), len(word_position_list[0])

(369, 369)

In [10]:
class_vals = np.zeros(len(df_train))
c1,c2 = df_train.label.unique()[0],df_train.label.unique()[1]
class_vals[df_train.label==c1] = 1
class_vals[df_train.label==c2] = 2
class_vals[:4]

array([1., 1., 1., 1.])

### Embeding

Отдельное обучение
- word to vec + PCA
- BERT
- word2vec + Simple ML model

In [11]:
# t_all_w = Vocabulary(text_all_words_list)
# len(text_all_words_list),len(t_all_w)

#### Word to digit

In [12]:
# words_dicts = {}
# text_list_of_encoded_words = []
# for text in text_list_of_words:
#     current_encoding = []
#     for w in text:
#         if w not in words_dicts.keys():
#             words_dicts[w] = len(words_dicts)
#         current_encoding.append(words_dicts[w])
#     text_list_of_encoded_words.append(current_encoding)

# text_list_of_encoded_words[1][0:20]

### Dataset

In [13]:
seq_size = 2000
for i in range(len(text_list_of_words)):
    size = len(text_list_of_words[i])
    for _ in range(size,seq_size):
        text_list_of_words[i].append("")


seq_lens = [len(text_list_of_words[i]) for i in range(len(text_list_of_words))]
sum(seq_lens)/len(text_list_of_words)

2000.0

In [28]:
converted_words = {}

In [41]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

def embed_bert_cls(text):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0]#.cpu().numpy()


def get_converted_embedding(word):
  if not( word in converted_words.keys() ):
    converted_words[word] = embed_bert_cls(word)
  return converted_words[word]

def get_text_embedding(text):
    text_embedding = []
    for w in text:
        w_embedding = get_converted_embedding(w)
        text_embedding.append(w_embedding)
    #text_embedding = np.array(text_embedding)
    return torch.stack(text_embedding)

text_embedding_list = []
for text in tqdm(text_list_of_words[0:700]):
    text_embedding_list.append(get_text_embedding(text).cpu().numpy())


Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 700/700 [00:03<00:00, 231.60it/s]


In [22]:
embed_bert_cls(["слово","слово","слово"]).shape

torch.Size([312])

In [58]:
class BERTDataset(Dataset):
    def __init__(self, text_embedding_list,word_position_list,class_vals):
        self.text_embedding_list = text_embedding_list
        self.word_position_list = word_position_list
        self.class_val = class_vals
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        #self.tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
        #self.model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
        #self.model.to(self.device)

    def __len__(self):
        return len(self.text_embedding_list)
    
    # def embed_bert_cls(self,text):
        
        
    #     t = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    #     with torch.no_grad():
    #         model_output = self.model(**{k: v.to(model.device) for k, v in t.items()})
    #     embeddings = model_output.last_hidden_state[:, 0, :]
    #     embeddings = torch.nn.functional.normalize(embeddings)
    #     return embeddings[0]#.cpu().numpy()
    
    # def get_text_embedding(self,text):
    #     text_embedding = []
    #     for w in text:
    #         w_embedding = self.embed_bert_cls(w)
    #         text_embedding.append(w_embedding)
    #     #text_embedding = np.array(text_embedding)
    #     return torch.stack(text_embedding)
    
    def __getitem__(self, idx):
        text_embedding = torch.tensor(text_embedding_list[idx])
        
        label = torch.tensor(word_position_list[idx])
        
        start,end = word_position_list[idx][0],word_position_list[idx][1]
        label_seq = torch.zeros(2000)
        if (start!=end):
            label_seq[start:end] = 1
        
        class_code = torch.tensor([0.,1.]) if self.class_val[idx]==1 else torch.tensor([1.,0.])
        
        text_embedding = text_embedding.to(self.device)
        label_seq = label_seq.to(self.device)
        class_code = class_code.to(self.device)
        return text_embedding,label_seq, class_code#,label

In [59]:
train_idx,test_idx = train_test_split(np.arange(len(text_embedding_list)), test_size=0.2, random_state=42)

In [61]:
batch_size = 100
#Data
train_data = BERTDataset(list(pd.Series(text_embedding_list)[train_idx]),
                        list(pd.Series(output_word_position_list)[train_idx]),
                        list(pd.Series(class_vals)[train_idx]))
test_data = BERTDataset(list(pd.Series(text_embedding_list)[test_idx]),
                        list(pd.Series(output_word_position_list)[test_idx]),
                        list(
                            pd.Series(class_vals)[test_idx]))

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

## Model

In [71]:
class LSTMModel(torch.nn.Module):

    def __init__(self):
        super(LSTMModel, self).__init__()
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        self.lstm1 = torch.nn.LSTM(input_size =312,hidden_size = 3,num_layers = 1)
        self.linear1 = torch.nn.Linear(2, 2000*3)
        
        self.lstm2 = torch.nn.LSTM(input_size = 3,hidden_size = 1,num_layers = 2)
        
        self.linear2 = torch.nn.Linear(2000, 2000)
        
        self.softmax = torch.nn.Softmax()

    def forward(self, seq,c):
        
        class_seq = self.linear1(c)
        
        h_0 = torch.autograd.Variable(torch.zeros(1, 2000, 3).to(self.device))
        c_0 = torch.autograd.Variable(torch.zeros(1, 2000, 3).to(self.device))

        output, (final_hidden_state, final_cell_state) = self.lstm1(seq, (h_0, c_0))
        output = output.reshape(100,-1) + class_seq
        output = output.reshape(2000,3)
        
        h_0 = torch.autograd.Variable(torch.zeros(1, batch_size, 2000).to(self.device))
        c_0 = torch.autograd.Variable(torch.zeros(1, batch_size, 2000).to(self.device))

        output, (final_hidden_state, final_cell_state) = self.lstm2(output, (h_0, c_0))
        output = self.linear2(output)
        return self.softmax(output)

In [72]:
#Train
def train(model,dataloader,loss,opt,train_size):
    model.train()
    current_loss = 0
    current_acc = 0
    
    for step, (text,label_seq,class_code) in enumerate(dataloader):
        res = model(text,class_code)

        l_val = loss(res, label_seq)
        current_loss += l_val
        current_acc += (res==label_seq).sum()

        l_val.backward()
        opt.step()

    current_loss /= train_size
    current_acc /= train_size
    
    return current_loss, current_acc
    

In [73]:
#Train
def test(model,dataloader,loss,test_size):
    model.eval()
    current_loss = 0
    current_acc = 0
    for step, (text,label_seq,class_code) in enumerate(dataloader):
        res = model(text,class_code)

        l_val = loss(res, label_seq)
        current_loss += l_val
        current_acc += (res==label_seq).sum()

    current_loss /= test_size
    current_acc /= test_size
    
    return current_loss, current_acc
    

In [74]:
# %%capture --no-display
device = "cuda" if torch.cuda.is_available() else "cpu"
model = LSTMModel()
model = model.to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.7, 0.899))
loss = torch.nn.MSELoss()
epoch = 10

loss_vals = [[],[]]
acc = [[],[]]

for i in tqdm(range(epoch)):
    
    train_l,train_acc = train(model,train_dataloader,loss,opt,train_size=len(train_idx))
    test_l,test_acc = test(model,test_dataloader,loss,test_size=len(test_idx))

    loss_vals[0].append(train_l)
    loss_vals[1].append(test_l)

    acc[0].append(train_acc)
    acc[1].append(test_acc)
    #clear_output(wait=True)
    

  0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: ignored